In [1]:
import json
import urllib.request
from urllib.error import HTTPError
import time
import math

In [2]:
franchise = 'scottgames'
franchise_name = 'fnaf'
page_limit = 30

franchises_file = '../wildcards/Franchises.yaml'
franchises_text = """
'<[model][illu]>source_anime, __Characters/${FRANCHISE}__**(yellow skin, orange skin, red skin:1.5)**':
  Description:
    - ${FRANCHISE} Characters
  Tags:
    - Characters
    - ${FRANCHISE}
"""
characterFile = f'../wildcards/characters/{franchise_name}.txt'

In [3]:
def myurlopen(url, max_wait=300):
    i = 2
    while True:
        try:
            req = urllib.request.Request(
                                            url,
                                            data=None,
                                            headers={
                                                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0'
                                            }
                                        )
            r = urllib.request.urlopen(req)
            return r
        except HTTPError as e:
            print(e)
            sleep_time = 2 + 2**i / 1000
            if sleep_time > max_wait:
                print('ABORT!')
                raise TimeoutError(f'Timed out with max wait time of {max_wait} s')
            print(f'Some HTTP error occurred. Waiting for {sleep_time}...')
            time.sleep(sleep_time)
            i += 1
            continue


def cosine_similarity(tag1_count, tag2_count, tag12_count):
    return tag12_count / math.sqrt(tag1_count * tag2_count)

def get_e621_count(search_string):
    url = f"https://search.yiff.today/count?query={search_string}"
    print(f'now working on url: {url}')
    response = myurlopen(url)
    return json.loads(response.read())['count']

def get_similarity(tag1, tag2):
    tag1_count = get_e621_count(tag1)
    #tag2_count = get_e621_count(tag2)
    combined_count = get_e621_count(f'{tag1}+{tag2}')
    #return cosine_similarity(tag1_count, tag2_count, combined_count)
    return combined_count / tag1_count

In [4]:
jdata = []
for page in range(1, page_limit):
    url = f"https://e621.net/posts.json?limit=320&tags={franchise}&page={page}"
    print(f'now working on url: {url}')
    response = myurlopen(url)
    rjson = json.loads(response.read())
    if not rjson['posts']:
        break
    jdata += rjson['posts']
    time.sleep(1)

now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=1
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=2
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=3
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=4
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=5
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=6
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=7
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=8
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=9
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=10
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=11
now working on url: https://e621.net/posts.json?limit=320&tags=scottgames&page=12
now working on url: https

In [5]:
character_tag_dict = {}

for post in jdata:
    post_tags = post['tags']
    characters = post_tags['character']

    for c in characters:
        if c in character_tag_dict.keys():
            character_tag_dict[c] += 1
        else:
            character_tag_dict[c] = 1

In [6]:
reject_if_below_threshold = 0.7
reject_if_above_threshold = 0.6
franchise_overlap_threshold = 0.8
total_post_threshold = 100
tag_overlap = 0.66

In [7]:
filtered_dict = {}

for c in character_tag_dict.keys():
    if character_tag_dict[c] < total_post_threshold:
        continue
    filtered_dict[c] = character_tag_dict[c]

In [8]:
accept_criteria = [
    'female',
    '1girl',
    '1girls'
]

reject_criteria = [
    'male',
    '1boy'
]

In [9]:
accepted_characters = []
needs_swap = []

for c in filtered_dict.keys():
    hits = 0
    similarity = get_similarity(c, franchise)
    if similarity < franchise_overlap_threshold:
        continue
    for a in accept_criteria:
        similarity = get_similarity(c, a)
        if similarity < reject_if_below_threshold:
            hits += 1
    for a in reject_criteria:
        similarity = get_similarity(c, a)
        if similarity > reject_if_above_threshold:
            hits += 1
            needs_swap.append(c)
    if hits == 0:
        accepted_characters.append(c)

now working on url: https://search.yiff.today/count?query=roxanne_wolf
now working on url: https://search.yiff.today/count?query=roxanne_wolf+scottgames
now working on url: https://search.yiff.today/count?query=roxanne_wolf
now working on url: https://search.yiff.today/count?query=roxanne_wolf+female
now working on url: https://search.yiff.today/count?query=roxanne_wolf
now working on url: https://search.yiff.today/count?query=roxanne_wolf+1girl
now working on url: https://search.yiff.today/count?query=roxanne_wolf
now working on url: https://search.yiff.today/count?query=roxanne_wolf+1girls
now working on url: https://search.yiff.today/count?query=roxanne_wolf
now working on url: https://search.yiff.today/count?query=roxanne_wolf+male
now working on url: https://search.yiff.today/count?query=roxanne_wolf
now working on url: https://search.yiff.today/count?query=roxanne_wolf+1boy
now working on url: https://search.yiff.today/count?query=montgomery_gator
now working on url: https://sear

In [10]:
accepted_characters

['roxanne_wolf',
 'toy_chica_(fnaf)',
 'cupcake_(fnaf)',
 'chica_(fnaf)',
 'vanny_(fnaf)',
 'mangle_(fnaf)',
 'bonfie_(cryptiacurves)',
 'fexa_(cryptiacurves)',
 'glamrock_chica',
 'chiku_(cryptiacurves)',
 'frenni_(cryptiacurves)',
 'lovetaste_chica',
 'runaboo_chica',
 'withered_chica_(fnaf)']

In [15]:
needs_swap = set(needs_swap)
needs_swap

{'bonnie_(fnaf)',
 'foxy_(fnaf)',
 'freddy_(fnaf)',
 'glamrock_bonnie_(fnaf)',
 'glamrock_freddy',
 'golden_freddy_(fnaf)',
 'montgomery_gator',
 'spring_bonnie_(fnaf)',
 'springtrap_(fnaf)',
 'toy_bonnie_(fnaf)',
 'toy_freddy_(fnaf)',
 'withered_bonnie_(fnaf)'}

In [11]:
characters = []

for c in accepted_characters:

    character = c.strip().replace('_', ' ').replace('(', '\\(').replace(')', '\\)')
    the_franchise = franchise.strip().replace('_', ' ').replace('(', '\\(').replace(')', '\\)')
    final_string = f'({character}:1.3), {the_franchise}\n'
    characters.append(final_string)

In [16]:
for c in needs_swap:

    character = c.strip().replace('_', ' ').replace('(', '\\(').replace(')', '\\)')
    the_franchise = franchise.strip().replace('_', ' ').replace('(', '\\(').replace(')', '\\)')
    final_string = f'({character}:1.3), {the_franchise}, (genderswap:1.5)\n'
    characters.append(final_string)

In [ ]:
if len(characters) > 0:
    with open(characterFile, 'a+') as f:
        f.writelines( characters )
    with open(franchises_file, 'a+') as f:
        new_text = franchises_text.replace('${FRANCHISE}', franchise_name)
        f.write(new_text)